<a href="https://colab.research.google.com/github/Safalgautam3636/Starting-/blob/master/Colorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from keras.layers import Conv2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave,imshow
import numpy as np
import tensorflow as tf
import sklearn
from keras.preprocessing.image import img_to_array, load_img
from keras.layers import UpSampling2D
import os
import tensorboard
import matplotlib.pyplot  as plt
import matplotlib.image as mpimg


In [6]:
path = '/content/Untitled Folder'
#Normalizing
scaling_data= ImageDataGenerator(rescale=1. / 256)
#Resizing the image
train =scaling_data.flow_from_directory(path, target_size=(256, 256), batch_size=190, class_mode=None)

#Convert from RGB to Lab
#Lets make it more simplifies we put the L-channel into X
#We put a-b channel into Y channel

X =[]
Y =[]
for img in train[0]:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128)
      #Here we are dividing the Y list value by 128 because we have to make that in range of[1,-1] since the 128 varies upto negativity -128 of a-b channel

X = np.array(X)#convert into numpy array

Y = np.array(Y)#convert into numpy array making ready to train
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y'''

print(X.shape)
print(Y.shape)
  

Found 190 images belonging to 2 classes.
(190, 256, 256, 1)
(190, 256, 256, 2)


In [7]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
#Decoding 
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))#we use hyperbolic function because the range f a,b varries from -1,to 1
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.fit(X,Y,validation_split=0.1, epochs=5, batch_size=8)
model.save('content/testfile.model')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 512)       1180160   
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 512)      

In [56]:
tf.keras.models.load_model(
    'content/testfile.model',
    custom_objects=None,
    compile=True)
im=[]
img1=img_to_array(load_img('images/sunset.png'))
img1 = resize(img1 ,(256,256))
im.append(img1)
im= np.array(img1_color, dtype=float)
im = rgb2lab(1.0/255*img1_color)[:,:,:,0]
im= im.reshape(img1_color.shape+(1,))
output1 = model.predict(img1_color)
output1 = output1*128
result = np.zeros((256, 256, 3))
result[:,:,0] = im[0][:,:,0]
result[:,:,1:] = output1[0]

OSError: ignored